In [56]:
import re
import requests
from bs4 import BeautifulSoup
import json

# https://codebeautify.org/htmlviewer/#
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-the-tree

In [57]:
persons_names = page.body.find_all("div", title=True)

In [ ]:
for person in persons_names:
    title = person["title"]
    data_persons[title] = {}

In [ ]:
page.body.find_all("div", class_="post person")

In [60]:
persons = page.body.find_all("div", class_="post person")

In [61]:
def get_languages(link):
    url = "https://www.hse.ru" + link
    p = requests.get(url)
    person_page = BeautifulSoup(p.text, 'html.parser')
    
    # Владение языками
    try: 
        lang_list = person_page.body.find("dl", class_="main-list large main-list-language-knowledge-level").find_all("dd")
        lang_list = [lang.text for lang in lang_list]
    except AttributeError:
        lang_list = 'none'
    
    return lang_list
    
#get_languages('/org/persons/25477')

In [ ]:
def get_contacts(link):
    url = "https://www.hse.ru" + link
    p = requests.get(url)
    person_page = BeautifulSoup(p.text, 'html.parser')
    contacts = {
        'phone_list': [],
        #'mail_list': [],
        'address': []
    }
    
    contacts_box = person_page.body.find("dl", class_="main-list large")
    #print(contacts_box.prettify())
    try:
        contents = contacts_box.find_all("dd")
        for box in contents: 
            if "Телефон" in box.text:
                phone_list = box.text.split(":")[1:]
                contacts['phone_list'] = phone_list

            elif "Электронная почта" in box.text:
                #mail_list = ['mail']
                #contacts['mail_list'] = mail_list
                continue

            elif "Адрес" in box.text:
                address = box.contents[0].split(":")[-1].strip()
                contacts['address'] = address
    except AttributeError:
        pass        
        
    return contacts


In [ ]:
def get_science_info(link):
    url = "https://www.hse.ru" + link
    p = requests.get(url)
    person_page = BeautifulSoup(p.text, 'html.parser')
    ids = {}
    
    extra_info = person_page.find("dl", class_="main-list person-extra-indent")
    try:
        ids_info = extra_info.find_all('dd')
    except AttributeError:
        advisor_id = 'none'
        return ids, advisor_id
    
    for dd in ids_info:
        try: 
            id_name = dd.span.text
            id_number = dd.a.text
            ids[id_name] = id_number
        except AttributeError:
            continue
    
    try:
        advisor_tag = person_page.find("dl", class_="main-list person-extra-indent").next_sibling
        advisor_id = advisor_tag.a['href']
    except TypeError:
        advisor_id = 'none'
    
    return ids, advisor_id
    

In [63]:
def get_intersts(link):
    url = "https://www.hse.ru" + link
    p = requests.get(url)
    person_page = BeautifulSoup(p.text, 'html.parser')
    interests = {}
    
    try:
        interests_div = person_page.find("div", class_="b-person-data__inner b-person-data__tags")
        interests_tags = interests_div.find_all('a')
        for a in interests_tags:
            interest_name = a.text
            interest_id = a['href'].split("=")[1]
            interests[interest_name] = interest_id
    except AttributeError:
        pass
        
    return interests
    
get_intersts('/staff/aparinov')

{'алгоритмы': '71848768',
 'анализ данных': '61913648',
 'анализ формальных понятий': '61296475'}

In [ ]:
with open("persons_Б.txt", 'w', encoding="utf-8") as outfile:
    for person in persons:
        name = person.find("div", title=True)["title"]
        link = person.find("a", href=True)["href"]

        lang_list = get_languages(link)
        contacts_dict = get_contacts(link)
        science_dict, advisor = get_science_info(link)
        interests_dict = get_intersts(link)


        data_persons[name] = {
            "link": link,
            "languages": lang_list,
            "contacts": contacts_dict,
            "science_ids": science_dict,
            "advisor": advisor,
            "interests": interests_dict


        }
        
        
    json.dump(data_persons, outfile, ensure_ascii=False)    


In [ ]:
with open("persons_A.txt", 'w', encoding="utf-8") as outfile:
    json.dump(data_persons, outfile, ensure_ascii=False)

In [66]:
letters_list = ["В", "Г", "Д", "Е", "Ж", "З", "И", "К", "Л", "М", "Н", "О", "П", "Р", "С", "Т",
                "У", "Ф", "Х", "Ц", "Ч", "Ш", "Щ", "Э", "Ю", "Я"]

In [ ]:
for letter in letters_list:
    link = "https://www.hse.ru/org/persons/?ltr=" + letter + ";udept=22726"
    data_persons = {}
    
    r = requests.get(link)
    page = BeautifulSoup(r.text, 'html.parser')
    persons_names = page.body.find_all("div", title=True)
    for person in persons_names:
        title = person["title"]
        data_persons[title] = {}
    
    persons = page.body.find_all("div", class_="post person")
    
    for person in persons:
        name = person.find("div", title=True)["title"]
        link = person.find("a", href=True)["href"]

        lang_list = get_languages(link)
        contacts_dict = get_contacts(link)
        science_dict, advisor = get_science_info(link)
        interests_dict = get_intersts(link)


        data_persons[name] = {
            "link": link,
            "languages": lang_list,
            "contacts": contacts_dict,
            "science_ids": science_dict,
            "advisor": advisor,
            "interests": interests_dict
        }
        
    with open("persons_"+letter+".txt", 'w', encoding="utf-8") as outfile:
        json.dump(data_persons, outfile, ensure_ascii=False)